# 2.- Euler-Maruyama method

Once we have reviewed basic concepts about probability and sampling distributions using `numpy`, we can delve into the 
technical core of this project. The objective of this notebook is to introduce the Euler-Maruyama (EM) method to numerically solve a Stochastic differential equation (SDE). Before presenting the numerical method, we provide a brief explanation of what is an SDE and what are the main components. We then describe the Brownian motion and its relationship with Gaussian normal distributions. The Brownian motion is a essential building block to understand and solve SDEs. We conclude with the EM method and how to implement it using `numpy` functionalities.

## Contents
### [A. Stochastic differential equations](#sde)
### [B. Brownian Motion](#brownian)
### [C. Euler-Maruyama method](#em)

<a id="sde"></a>
### A. Stochastic differential equations

An SDE is a mathematical equation describing the dynamics of a system or process subject to deterministic and random influences. It formulates the time-evolution equation of variables with inherent randomness. By seamlessly incorporating random terms, SDEs offer a powerful framework for analysing and understanding random variable behaviour. As a result, they find wide-ranging applicability in fields such as biology, finance and physics.

Although there are different types of SDEs, we restrict our attention to the following type of SDE, so-called Itô SDE. Considering that $X_t$ represents the variable we are interested at time $t$, its related SDE is written in the form of:

\begin{align}
dX_{t} = \mu(X_t, t)dt + \sigma(X_t, t) dW_t.
\end{align}

Now, we explain each component of the equation above.

* $dX_t$ is the differential of $X_t$, i.e., a small (infinitesimal) change of $X_t$. 
* $\mu(X_t, t)dt$ is the drift term. It represents the deterministic force driving the expected or average rate of change in $X_t$ at time $t$. In general, the drift coefficient $\mu(\cdot)$ can be a function of the process value $X_t$ and the time step $t$ as represented here or it can be a constant, $\mu(X_t, t) = \mu$. $dt$ represents the differential of time. As we will see later, this term informs about the trend behaviour of $X$ over time.
* $\sigma(X_t, t)dW_t$ is the diffusion term. It represents the stochastic part of the equation, introducing the random behaviour. The term $\sigma(X_t, t)$ is called the diffusion coefficient and it is inherently related to the volatility or uncertainty evolution of the variable $X_t$ at time $t$. Like the drift coefficient, the diffusion coefficient can be a function of the process value $X_t$ and be time-dependent $t$ or it can be a constant $\sigma(X_t, t) = \sigma$. $dW_t$ is a Wiener process, so-called Brownian motion. It involves the random fluctuation of the variable $X$ injecting random noise in the time-evolution equation. We will provide a detailed explanation of the Brownian motion and its implications in the next subsection. Now, we can think of the Brownian motion as a noise generator that follows a Gaussian distribution $W_t \sim \mathcal{N(0, t)}$.
